In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine


In [25]:

# Tabla CERVEZAS
cervezas = pd.DataFrame({
    'CodC': ['01', '02', '03', '04', '05'],
    'Envase': ['Botella', 'Botella', 'Lata', 'Botella', 'Barril'],
    'Capacidad': [0.2, 0.33, 0.33, 1, 60],
    'Stock': [3600, 1200, 2400, 288, 30]
})

# Tabla BARES
bares = pd.DataFrame({
    'CodB': ['001', '002', '003', '004'],
    'Nombre': ['Stop', 'Las Vegas', 'Club Social', 'Otra Ronda'],
    'Cif': ['11111111X', '22222222Y', '33333333Z', '44444444Z'],
    'Localidad': ['Villa Botijo', 'Villa Botijo', 'Las Ranas', 'La Esponja']
})

# Tabla EMPLEADOS
empleados = pd.DataFrame({
    'CodE': [1, 2, 3],
    'Nombre': ['Prudencio Caminero', 'Vicente Merario', 'Valentin Siempre'],
    'Sueldo': [120000, 110000, 100000]
})

# Tabla REPARTO
reparto = pd.DataFrame({
    'CodE': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3],
    'CodB': ['001', '001', '002', '004', '002', '002', '004', '004', '003', '003'],
    'CodC': ['01', '02', '03', '05', '03', '05', '01', '02', '03', '04'],
    'Fechas': ['2005-10-21', '2005-10-21', '2005-10-22', '2005-10-22', '2005-10-22', '2005-10-23','2005-10-23', '2005-10-24', '2005-10-24', '2005-10-25'],
    'Cantidad': [240, 48, 60, 4, 48, 2, 480, 72, 48,20]
})

In [26]:
churro = "sqlite:///BASE_BARES.db"
engine = create_engine(churro)
conn = sqlite3.connect("BASE_BARES.db")


cervezas.to_sql("cervezas", con=engine, if_exists="replace", index=None)
bares.to_sql("bares", con=engine, if_exists="replace", index=None)
empleados.to_sql("empleados", con=engine, if_exists="replace", index=None)
reparto.to_sql("reparto", con=engine, if_exists="replace", index=None)

10

### Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana del 17 al 23 de octubre de 2005. 

In [27]:
query = """ 
SELECT * FROM (SELECT * FROM (SELECT *
FROM empleados
JOIN reparto
ON empleados.CodE = reparto.CodE) AS tabla1
JOIN bares
WHERE Fechas BETWEEN "10/17/05" AND "10/23/05") AS tabla2
WHERE CodB = "001" and "Nombre:1" = "Stop"
"""


pd.read_sql_query(query, con=engine)

,CodE,Nombre,Sueldo,CodE:1,CodB,CodC,Fechas,Cantidad,CodB:1,Nombre:1,Cif,Localidad


### Obtener  el   Cif  y  nombre  de  los  bares  a  los  que  se  ha  repartido  cerveza  de  tipo  Botella  y  capacidad inferior a 1 litro, ordenados por localidad 

In [28]:
query = """ 
SELECT * FROM(SELECT * FROM(SELECT * FROM (SELECT Cif, nombre
FROM bares) AS Bares
JOIN cervezas)AS Cervezas
Where Envase = "Botella" AND Capacidad < 1) AS Ultima

"""

pd.read_sql_query(query, con=engine)



,Cif,nombre,CodC,Envase,Capacidad,Stock
0,11111111X,Stop,01,Botella,0.20,3600
1,11111111X,Stop,02,Botella,0.33,1200
2,22222222Y,Las Vegas,01,Botella,0.20,3600
3,22222222Y,Las Vegas,02,Botella,0.33,1200
4,33333333Z,Club Social,01,Botella,0.20,3600
5,33333333Z,Club Social,02,Botella,0.33,1200
6,44444444Z,Otra Ronda,01,Botella,0.20,3600
7,44444444Z,Otra Ronda,02,Botella,0.33,1200


### Obtener  los  repartos  (nombre  del  bar,  envase  y  capacidad  de  la  bebida, fecha y cantidad) realizados por Prudencio Camionero.

In [29]:
query = """ 
SELECT DISTINCT empleados.Nombre AS "Nombre Empleado",
bares.Nombre AS Nombre_bar,
cervezas.Envase,
cervezas.Capacidad,
reparto.Fechas
FROM reparto JOIN bares
ON bares.CodB = reparto.CodB

JOIN cervezas
ON cervezas.CodC = reparto.CodC

JOIN empleados 
ON empleados.CodE = reparto.CodE

WHERE "Nombre Empleado" = "Prudencio Caminero"

"""

pd.read_sql_query(query, con=engine)

,Nombre Empleado,Nombre_bar,Envase,Capacidad,Fechas
0,Prudencio Caminero,Stop,Botella,0.20,2005-10-21
1,Prudencio Caminero,Stop,Botella,0.33,2005-10-21
2,Prudencio Caminero,Las Vegas,Lata,0.33,2005-10-22
3,Prudencio Caminero,Otra Ronda,Barril,60.00,2005-10-22


### 4: Obtener  los  bares  a  los  que  se  les  ha  repartido  envases  de  tipo  botella  y  capacidad  0.2  ó  0.33

In [31]:
query = """ 
SELECT DISTINCT bares.Nombre,
cervezas.Envase,
cervezas.Capacidad

FROM bares JOIN cervezas 

WHERE Capacidad = "0.20" OR Capacidad = "0.33" and Envase = "Botella"

"""

pd.read_sql_query(query, con=engine)

,Nombre,Envase,Capacidad
0,Stop,Botella,0.20
1,Las Vegas,Botella,0.20
2,Club Social,Botella,0.20
3,Otra Ronda,Botella,0.20
4,Stop,Botella,0.33
5,Las Vegas,Botella,0.33
6,Club Social,Botella,0.33
7,Otra Ronda,Botella,0.33


In [52]:
# Tabla CERVEZAS
cervezas = pd.DataFrame({
    'CodC': ['01', '02', '03', '04', '05'],
    'Envase': ['Botella', 'Botella', 'Lata', 'Botella', 'Barril'],
    'Capacidad': [0.2, 0.33, 0.33, 1, 60],
    'Stock': [3600, 1200, 2400, 288, 30]
})

# Tabla BARES
bares = pd.DataFrame({
    'CodB': ['001', '002', '003', '004'],
    'Nombre': ['Stop', 'Las Vegas', 'Club Social', 'Otra Ronda'],
    'Cif': ['11111111X', '22222222Y', '33333333Z', '44444444Z'],
    'Localidad': ['Villa Botijo', 'Villa Botijo', 'Las Ranas', 'La Esponja']
})

# Tabla EMPLEADOS
empleados = pd.DataFrame({
    'CodE': [1, 2, 3],
    'Nombre': ['Prudencio Caminero', 'Vicente Merario', 'Valentin Siempre'],
    'Sueldo': [120000, 110000, 100000]
})

# Tabla REPARTO
reparto = pd.DataFrame({
    'CodE': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3],
    'CodB': ['001', '001', '002', '004', '002', '002', '004', '004', '003', '003'],
    'CodC': ['01', '02', '03', '05', '03', '05', '01', '02', '03', '04'],
    'Fechas': ['2005-10-21', '2005-10-21', '2005-10-22', '2005-10-22', '2005-10-22', '2005-10-23','2005-10-23', '2005-10-24', '2005-10-24', '2005-10-25'],
    'Cantidad': [240, 48, 60, 4, 48, 2, 480, 72, 48,20]})

### 5: Nombre de los empleados que han repartido a los bares "Stop" y "Las Vegas" cervezas con envase botella.

In [58]:
query = """ 
SELECT empleados.Nombre AS Empleado,
bares.Nombre AS Bar,
cervezas.Envase

FROM empleados JOIN reparto ON empleados.CodE = reparto.CodE
JOIN bares ON bares.CodB = reparto.CodB
JOIN cervezas ON cervezas.CodC = reparto.CodC

WHERE Envase = "Botella"

"""

pd.read_sql_query(query, con=engine )

,Empleado,Bar,Envase
0,Prudencio Caminero,Stop,Botella
1,Vicente Merario,Otra Ronda,Botella
2,Prudencio Caminero,Stop,Botella
3,Vicente Merario,Otra Ronda,Botella
4,Valentin Siempre,Club Social,Botella
